In [1]:
%pip install -q langchain langchain-community langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set NVIDIA API key from environment variable
# If not found in environment, this will be None
api_key = os.environ.get("NVIDIA_API_KEY")

# Print a message based on whether the key was found
if api_key:
    print("NVIDIA API key loaded successfully")
else:
    raise ValueError("NVIDIA API key not found in .env file. Please set NVIDIA_API_KEY in your environment variables.")


NVIDIA API key loaded successfully


In [3]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Get all available models
all_models = ChatNVIDIA.get_available_models()

# Display only the first 20 models
print(f"First 20 of {len(all_models)} available models:")
for i, model in enumerate(all_models[:20]):
    print(f"{i+1}. {model}")

First 20 of 88 available models:
1. id='institute-of-science-tokyo/llama-3.1-swallow-70b-instruct-v0.1' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=True base_model=None
2. id='nvidia/llama-3.1-nemotron-51b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
3. id='qwen/qwen2.5-coder-32b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
4. id='google/gemma-7b' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=['ai-gemma-7b', 'playground_gemma_7b', 'gemma_7b'] supports_tools=False supports_structured_output=False base_model=None
5. id='yentinglin/llama-3-taiwan-70b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=False supports_structured_output=False base_model=None
6. id='microsoft/k

In [4]:
###################################### Document Summarization  ##################################

from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader, ArxivLoader

from pydantic import BaseModel, Field
from typing import List
from IPython.display import clear_output
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

# Define the Pydantic model for document summarization
class DocumentSummaryBase(BaseModel):
    """Base class for document summarization results."""
    running_summary: str = Field(default="", description="The running summary of the document. Update, don't override.")
    main_ideas: List[str] = Field(default_factory=list, description="Maximum 3 important points from the document.")
    loose_ends: List[str] = Field(default_factory=list, description="Maximum 3 open questions or areas needing clarification.")

# Set up console formatting with NVIDIA brand color
console = Console()
nvidia_green = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=nvidia_green)

# Document Loading 
# For local files (commented out as an option)
# loader = UnstructuredFileLoader("your_document.pdf")
# docs = loader.load()

# Load document from Arxiv (GraphRAG paper)
loader = ArxivLoader(query="2404.16130")  # GraphRAG paper
docs = loader.load()

# Configure text splitter for document chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)

# Optional preprocessing (commented out)
# def preprocess_text(text):
#     # Remove special characters, normalize whitespace, etc.
#     return text.replace("...", ".")

# Split documents into manageable chunks
docs_split = text_splitter.split_documents(docs)
print(f"Document split into {len(docs_split)} chunks")

# Create summarization prompt
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    You are a technical document summarizer tasked with generating a running summary of a research document.
    You are given chunks of text from a document and an existing knowledge base containing:
    1. A running summary of the document so far
    2. Main ideas (max 3)
    3. Loose ends or questions (max 3)
    
    Your goal is to update this knowledge base with new information from the current document chunk.
    
    IMPORTANT INSTRUCTIONS:
    - DO NOT lose information when updating the knowledge base
    - Integrate new information with existing running_summary
    - Update main_ideas and loose_ends as needed, but keep them limited to 3 items each
    - Follow the format instructions exactly
    - If nothing new is present in the chunk, return the existing knowledge base unchanged
    
    {format_instructions}
    """),
    ("user", "{input}")
])




Document split into 83 chunks


In [5]:
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace(r"\]", "]")
            .replace(r"\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser


In [6]:
latest_summary = ""

def RSummarizer(knowledge, llm, prompt, verbose=False):
    def summarize_docs(docs):
        parse_chain = RunnableAssign({'info_base' : RExtract(knowledge.__class__, llm, prompt)})
        state = {'info_base' : knowledge}

        global latest_summary  ## If your loop crashes, you can check out the latest_summary

        for i, doc in enumerate(docs):
            state['input'] = doc.page_content
            state = parse_chain.invoke(state)

            assert 'info_base' in state
            if verbose:
                print(f"Considered {i+1} documents")
                pprint(state['info_base'])
                latest_summary = state['info_base']
                clear_output(wait=True)

        return state['info_base']
    return RunnableLambda(summarize_docs)

instruct_model = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.3").bind(max_tokens=4096)
instruct_llm = instruct_model | StrOutputParser()

## Take the first 10 document chunks and accumulate a DocumentSummaryBase
summarizer = RSummarizer(DocumentSummaryBase(), instruct_llm, summary_prompt, verbose=True)
summary = summarizer.invoke(docs_split[:10])

Considered 10 documents


DocumentSummaryBase(running_summary='', main_ideas=[], loose_ends=[])

In [ ]:
###################################### Retrieval-Augmented Generation (RAG) ##################################

import json
import gradio as gr
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader, UnstructuredPDFLoader
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from faiss import IndexFlatL2
from operator import itemgetter
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

## Data Loading/Text Splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2022-05-01',
    'Title': 'MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external 
knowledge sources and discrete reasoning',
    'Authors': 'Ehud Karpas, Omri Abend, Yonatan Belinkov, Barak Lenz, Opher Lieber, Nir Ratner, Yoav Shoham, Hofit
Bata, Yoav Levine, Kevin Leyton-Brown, Dor Muhlgay, Noam Rozen, Erez Schwartz, Gal Shachaf, Shai Shalev-Shwartz, 
Amnon Shashua, Moshe Tenenholtz',
    'Summary': 'Huge language models (LMs) have ushered in a new era for AI, serving as a\ngateway to 
natural-language-based knowledge tasks. Although an essential\nelement of modern AI, LMs are also inherently 
limited in a number of ways. We\ndiscuss these limitations and how they can be avoided by adopting a 
systems\napproach. Conceptualizing the challenge as one that involves knowledge and\nreasoning in addition to 
linguistic processing, we define a flexible\narchitecture with multiple neural models, complemented by discrete 
knowledge\nand reasoning modules. We describe this neuro-symbolic architecture, dubbed the\nModular Reasoning, 
Knowledge and Language (MRKL, pronounced "miracle") system,\nsome of the technical challenges in implementing it, 
and Jurassic-X, AI21 Labs\'\nMRKL system implementation.'
}


Document 4
 - # Chunks: 21
 - Metadata: 


{
    'Published': '2023-10-10',
    'Title': 'Mistral 7B',
    'Authors': 'Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, 
Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed',
    'Summary': 'We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered\nfor superior 
performance and efficiency. Mistral 7B outperforms Llama 2 13B\nacross all evaluated benchmarks, and Llama 1 34B in
reasoning, mathematics, and\ncode generation. Our model leverages grouped-query attention (GQA) for 
faster\ninference, coupled with sliding window attention (SWA) to effectively handle\nsequences of arbitrary length
with a reduced inference cost. We also provide a\nmodel fine-tuned to follow instructions, Mistral 7B -- Instruct, 
that surpasses\nthe Llama 2 13B -- Chat model both on human and automated benchmarks. Our\nmodels are released 
under the Apache 2.0 license.'
}


Document 5
 - # Chunks: 44
 - Metadata: 


{
    'Published': '2023-12-24',
    'Title': 'Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena',
    'Authors': 'Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang, Zi Lin, 
Zhuohan Li, Dacheng Li, Eric P. Xing, Hao Zhang, Joseph E. Gonzalez, Ion Stoica',
    'Summary': 'Evaluating large language model (LLM) based chat assistants is challenging\ndue to their broad 
capabilities and the inadequacy of existing benchmarks in\nmeasuring human preferences. To address this, we explore
using strong LLMs as\njudges to evaluate these models on more open-ended questions. We examine the\nusage and 
limitations of LLM-as-a-judge, including position, verbosity, and\nself-enhancement biases, as well as limited 
reasoning ability, and propose\nsolutions to mitigate some of them. We then verify the agreement between 
LLM\njudges and human preferences by introducing two benchmarks: MT-bench, a\nmulti-turn question set; and Chatbot 
Arena, a crowdsourced battle platform. Our\nresults reveal that strong LLM judges like GPT-4 can match both 
controlled and\ncrowdsourced human preferences well, achieving over 80% agreement, the same\nlevel of agreement 
between humans. Hence, LLM-as-a-judge is a scalable and\nexplainable way to approximate human preferences, which 
are otherwise very\nexpensive to obtain. Additionally, we show our benchmark and traditional\nbenchmarks complement
each other by evaluating several variants of LLaMA and\nVicuna. The MT-bench questions, 3K expert votes, and 30K 
conversations with\nhuman preferences are publicly available 
at\nhttps://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge.'
}

In [ ]:
%%time
## Constructing Your Document Vector Stores
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructing Vector Stores
Constructed aggregate docstore with 238 chunks
CPU times: user 557 ms, sys: 30.6 ms, total: 588 ms
Wall time: 16.6 s


In [ ]:
# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

## Implement the Stream Chain
stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

## Implement the RAG Chain
retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about RAG!"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me about RAG!',
    'history': '',
    'context': '[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . We refer to this 
decoding procedure as \\u201cThorough Decoding.\\u201d For longer\\noutput sequences, |Y | can become large, 
requiring many forward passes. For more ef\\ufb01cient decoding,\\nwe can make a further approximation that 
p\\u03b8(y|x, zi) \\u22480 where y was not generated during beam\\nsearch from x, zi. This avoids the need to run 
additional forward passes once the candidate set Y has\\nbeen generated. We refer to this decoding procedure as 
\\u201cFast Decoding.\\u201d\\n3\\nExperiments\\nWe experiment with RAG in a wide range of knowledge-intensive 
tasks. For all experiments, we use\\na single Wikipedia dump for our non-parametric knowledge source. Following Lee
et al. [31] and\\nKarpukhin et al. [26], we use the December 2018 dump. Each Wikipedia article is split into 
disjoint\\n100-word chunks, to make a total of 21M documents\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . Since RAG can be\\nemployed as a language model, similar concerns as for GPT-2 
[50] are valid here, although arguably\\nto a lesser extent, including that it might be used to generate abuse, 
faked or misleading content in\\nthe news or on social media; to impersonate others; or to automate the production 
of spam/phishing\\ncontent [54]. Advanced language models may also lead to the automation of various jobs in 
the\\ncoming decades [16]. In order to mitigate these risks, AI systems could be employed to \\ufb01ght 
against\\nmisleading content and automated spam/phishing.\\nAcknowledgments\\nThe authors would like to thank the 
reviewers for their thoughtful and constructive feedback on this\\npaper, as well as HuggingFace for their help in 
open-sourcing code to run RAG models. The authors\\nwould also like to thank Kyunghyun Cho and Sewon Min for 
productive discussions and advice. EP\\nthanks supports from the NSF Graduate Research Fellowship. PL is supported 
by the FAIR PhD\\nprogram.\\n\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] 
.\\nRAG-Token\\nThe RAG-Token model can be seen as a standard, autoregressive seq2seq genera-\\ntor with transition
probability: p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) = P\\nz\\u2208top-k(p(\\u00b7|x)) p\\u03b7(zi|x)p\\u03b8(yi|x, 
zi, y1:i\\u22121) To\\ndecode, we can plug p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) into a standard beam 
decoder.\\nRAG-Sequence\\nFor RAG-Sequence, the likelihood p(y|x) does not break into a conventional per-\\ntoken 
likelihood, hence we cannot solve it with a single beam search. Instead, we run beam search for\\neach document z, 
scoring each hypothesis using p\\u03b8(yi|x, z, y1:i\\u22121). This yields a set of hypotheses\\nY , some of which 
may not have appeared in the beams of all documents. To estimate the probability\\nof an hypothesis y we run an 
additional forward pass for each document z for which y does not\\nappear in the beam, multiply generator 
probability with p\\u03b7(z|x) and then sum the probabilities across\\nbeams for the marginals\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses 
the same document\\nto predict each target token. The second approach, RAG-Token, can predict each target token 
based\\non a different document. In the following, we formally introduce both models and then describe 
the\\np\\u03b7 and p\\u03b8 components, as well as the training and decoding 
procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same retrieved document to 
generate\\nthe complete sequence. Technically, it treats the retrieved document as a single latent variable 
that\\nis marginalized to get the seq2seq probability p(y|x) via a top-K approximation\n'
}

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about RAG!\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Retrieval-Augmented Generation for 
Knowledge-Intensive NLP Tasks] . We refer to this decoding procedure as \\u201cThorough Decoding.\\u201d For 
longer\\noutput sequences, |Y | can become large, requiring many forward passes. For more ef\\ufb01cient 
decoding,\\nwe can make a further approximation that p\\u03b8(y|x, zi) \\u22480 where y was not generated during 
beam\\nsearch from x, zi. This avoids the need to run additional forward passes once the candidate set Y has\\nbeen
generated. We refer to this decoding procedure as \\u201cFast Decoding.\\u201d\\n3\\nExperiments\\nWe experiment 
with RAG in a wide range of knowledge-intensive tasks. For all experiments, we use\\na single Wikipedia dump for 
our non-parametric knowledge source. Following Lee et al. [31] and\\nKarpukhin et al. [26], we use the December 
2018 dump. Each Wikipedia article is split into disjoint\\n100-word chunks, to make a total of 21M 
documents\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . Since RAG can 
be\\nemployed as a language model, similar concerns as for GPT-2 [50] are valid here, although arguably\\nto a 
lesser extent, including that it might be used to generate abuse, faked or misleading content in\\nthe news or on 
social media; to impersonate others; or to automate the production of spam/phishing\\ncontent [54]. Advanced 
language models may also lead to the automation of various jobs in the\\ncoming decades [16]. In order to mitigate 
these risks, AI systems could be employed to \\ufb01ght against\\nmisleading content and automated 
spam/phishing.\\nAcknowledgments\\nThe authors would like to thank the reviewers for their thoughtful and 
constructive feedback on this\\npaper, as well as HuggingFace for their help in open-sourcing code to run RAG 
models. The authors\\nwould also like to thank Kyunghyun Cho and Sewon Min for productive discussions and advice. 
EP\\nthanks supports from the NSF Graduate Research Fellowship. PL is supported by the FAIR 
PhD\\nprogram.\\n\n[Quote from Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] 
.\\nRAG-Token\\nThe RAG-Token model can be seen as a standard, autoregressive seq2seq genera-\\ntor with transition
probability: p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) = P\\nz\\u2208top-k(p(\\u00b7|x)) p\\u03b7(zi|x)p\\u03b8(yi|x, 
zi, y1:i\\u22121) To\\ndecode, we can plug p\\u2032\\n\\u03b8(yi|x, y1:i\\u22121) into a standard beam 
decoder.\\nRAG-Sequence\\nFor RAG-Sequence, the likelihood p(y|x) does not break into a conventional per-\\ntoken 
likelihood, hence we cannot solve it with a single beam search. Instead, we run beam search for\\neach document z, 
scoring each hypothesis using p\\u03b8(yi|x, z, y1:i\\u22121). This yields a set of hypotheses\\nY , some of which 
may not have appeared in the beams of all documents. To estimate the probability\\nof an hypothesis y we run an 
additional forward pass for each document z for which y does not\\nappear in the beam, multiply generator 
probability with p\\u03b7(z|x) and then sum the probabilities across\\nbeams for the marginals\n[Quote from 
Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks] . In one approach, RAG-Sequence, the model uses 
the same document\\nto predict each target token. The second approach, RAG-Token, can predict each target token 
based\\non a different document. In the following, we formally introduce both models and then describe 
the\\np\\u03b7 and p\\u03b8 components, as well as the training and decoding 
procedure.\\n2.1\\nModels\\nRAG-Sequence Model\\nThe RAG-Sequence model uses the same retrieved document to 
generate\\nthe c

RAG stands for Retrieval-Augmented Generation. It is a method that combines a retriever model, which identifies relevant documents from a large corpus, with a generator model, which produces answers to natural language processing (NLP) tasks. The retriever model is based on the DPR (Dense Passage Retrieval) model, and the generator model is based on a transformer model.

RAG can be employed in a variety of knowledge-intensive NLP tasks and is trained on a single Wikipedia dump, which is split into disjoint 100-word chunks to make a total of 21M documents.

There are two ways that RAG can be used, RAG-Sequence and RAG-Token. RAG-Sequence uses the same document to predict each target token, while RAG-Token can predict each target token based on a different document.

There are also two decoding procedures that can be used with RAG, Thorough Decoding and Fast Decoding. Thorough decoding runs additional forward passes for longer output sequences, while Fast Decoding makes an approximation 

In [ ]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

<ipython-input-28-7a9f48738b41>:1: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value = [[None, initial_msg]])
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://66d7f153f0437a03c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


{
    'input': 'Tell me about  BERT',
    'history': '[Quote from Document] User previously responded with Tell me about RAG!\n[Quote from Document] 
Agent previously responded with RAG stands for Retrieval-Augmented Generation. It is a method that combines a 
retriever model, which identifies relevant documents from a large corpus, with a generator model, which produces 
answers to natural language processing (NLP) tasks. The retriever model is based on the DPR (Dense Passage 
Retrieval) model, and the generator model is based on a transformer model.\n\nRAG can be employed in a variety of 
knowledge-intensive NLP tasks and is trained on a single Wikipedia dump, which is split into disjoint 100-word 
chunks to make a total of 21M documents.\n\nThere are two ways that RAG can be used, RAG-Sequence and RAG-Token. 
RAG-Sequence uses the same document to predict each target token, while RAG-Token can predict each target token 
based on a different document.\n\nThere are also two decoding procedures that can be used with RAG, Thorough 
Decoding and Fast Decoding. Thorough decoding runs additional forward passes for longer output sequences, while 
Fast Decoding makes an approximation that p(y|x,zi) ≈ 0 where y was not generated during the beam search from x, 
zi. This avoids the need to run additional forward passes once the candidate set Y has been generated.\n\nIt\'s 
important to note that similar concerns to GPT-2 apply to RAG, including the potential for misuse in generating 
abuse, faked or misleading content, impersonating others and automating the production of spam/phishing 
content.\nTherefore, AI systems should be employed to fight against misleading content and automated 
spam/phishing.\n\nThe RAG model was introduced in the paper "Retrieval-Augmented Generation for Knowledge-Intensive
NLP Tasks" (Lewis, Linford, et.al, 2020) and the documentation provided is from that paper.\n\n\nReferences:\n- 
Lewis, Parker, et al. "Retrieval-augmented generation for knowledge-intensive nlp tasks." arXiv preprint 
arXiv:2002.08913 (2020).\n- Radford, Alec, et al. "Language models are unsupervised multitask learners." OpenAI 
blog 1.8 (2019): 9.\n- Karpukhin, Vsevolod, et al. "Dense Passage Retrieval for Open-Domain Question Answering." 
arXiv preprint arXiv:2004.04906 (2020).\n',
    'context': '[Quote from BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding] 
.\\nBERT is conceptually simple and empirically\\npowerful.\\nIt obtains new state-of-the-art re-\\nsults on eleven
natural language processing\\ntasks, including pushing the GLUE score to\\n80.5% (7.7% point absolute 
improvement),\\nMultiNLI accuracy to 86.7% (4.6% absolute\\nimprovement), SQuAD v1.1 question answer-\\ning Test F1
to 93.2 (1.5 point absolute im-\\nprovement) and SQuAD v2.0 Test F1 to 83.1\\n(5.1 point absolute 
improvement).\\n1\\nIntroduction\\nLanguage model pre-training has been shown to\\nbe effective for improving many 
natural language\\nprocessing tasks (Dai and Le, 2015; Peters et al.,\\n2018a; Radford et al., 2018; Howard and 
Ruder,\\n2018). These include sentence-level tasks such as\\nnatural language inference (Bowman et al., 
2015;\\nWilliams et al\n[Quote from Document] {\'Published\': \'2019-05-24\', \'Title\': \'BERT: Pre-training of 
Deep Bidirectional Transformers for Language Understanding\', \'Authors\': \'Jacob Devlin, Ming-Wei Chang, Kenton 
Lee, Kristina Toutanova\', \'Summary\': \'We introduce a new language representation model called BERT, which 
stands\\nfor Bidirectional Encoder Representations from Transformers. Unlike recent\\nlanguage representation 
models, BERT is designed to pre-train deep\\nbidirectional representations from unlabeled text by jointly 
conditioning on\\nboth left and right context in all layers. As a result, the pre-trained BERT\\nmodel can be 
fine-tuned with just one additional output layer to create\\nstate-of-the-art models for a wide range of tasks, 
such as question answering

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about  BERT\n\n From this, we have retrieved the following potentially-useful info:  
Conversation History Retrieval:\n[Quote from Document] User previously responded with Tell me about RAG!\n[Quote 
from Document] Agent previously responded with RAG stands for Retrieval-Augmented Generation. It is a method that 
combines a retriever model, which identifies relevant documents from a large corpus, with a generator model, which 
produces answers to natural language processing (NLP) tasks. The retriever model is based on the DPR (Dense Passage
Retrieval) model, and the generator model is based on a transformer model.\n\nRAG can be employed in a variety of 
knowledge-intensive NLP tasks and is trained on a single Wikipedia dump, which is split into disjoint 100-word 
chunks to make a total of 21M documents.\n\nThere are two ways that RAG can be used, RAG-Sequence and RAG-Token. 
RAG-Sequence uses the same document to predict each target token, while RAG-Token can predict each target token 
based on a different document.\n\nThere are also two decoding procedures that can be used with RAG, Thorough 
Decoding and Fast Decoding. Thorough decoding runs additional forward passes for longer output sequences, while 
Fast Decoding makes an approximation that p(y|x,zi) ≈ 0 where y was not generated during the beam search from x, 
zi. This avoids the need to run additional forward passes once the candidate set Y has been generated.\n\nIt\'s 
important to note that similar concerns to GPT-2 apply to RAG, including the potential for misuse in generating 
abuse, faked or misleading content, impersonating others and automating the production of spam/phishing 
content.\nTherefore, AI systems should be employed to fight against misleading content and automated 
spam/phishing.\n\nThe RAG model was introduced in the paper "Retrieval-Augmented Generation for Knowledge-Intensive
NLP Tasks" (Lewis, Linford, et.al, 2020) and the documentation provided is from that paper.\n\n\nReferences:\n- 
Lewis, Parker, et al. "Retrieval-augmented generation for knowledge-intensive nlp tasks." arXiv preprint 
arXiv:2002.08913 (2020).\n- Radford, Alec, et al. "Language models are unsupervised multitask learners." OpenAI 
blog 1.8 (2019): 9.\n- Karpukhin, Vsevolod, et al. "Dense Passage Retrieval for Open-Domain Question Answering." 
arXiv preprint arXiv:2004.04906 (2020).\n\n\n Document Retrieval:\n[Quote from BERT: Pre-training of Deep 
Bidirectional Transformers for Language Understanding] .\\nBERT is conceptually simple and 
empirically\\npowerful.\\nIt obtains new state-of-the-art re-\\nsults on eleven natural language 
processing\\ntasks, including pushing the GLUE score to\\n80.5% (7.7% point absolute improvement),\\nMultiNLI 
accuracy to 86.7% (4.6% absolute\\nimprovement), SQuAD v1.1 question answer-\\ning Test F1 to 93.2 (1.5 point 
absolute im-\\nprovement) and SQuAD v2.0 Test F1 to 83.1\\n(5.1 point absolute 
improvement).\\n1\\nIntroduction\\nLanguage model pre-training has been shown to\\nbe effective for improving many 
natural language\\nprocessing tasks (Dai and Le, 2015; Peters et al.,\\n2018a; Radford et al., 2018; Howard and 
Ruder,\\n2018). These include sentence-level tasks such as\\nnatural language inference (Bowman et al., 
2015;\\nWilliams et al\n[Quote from Document] {\'Published\': \'2019-05-24\', \'Title\': \'BERT: Pre-training of 
Deep Bidirectional Transformers for Language Understanding\', \'Authors\': \'Jacob Devlin, Ming-Wei Chang, Kenton 
Lee, Kristina Toutanova\', \'Summary\': \'We introduce a new language representation model called BERT, which 
stands\\nfor Bidirectional Encoder Representations from Transformers. Unlike recent\\nlanguage representation 
models, BERT is designed to pre-train deep\\nbidirectional representations from unlabeled te

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://66d7f153f0437a03c7.gradio.live
Closing server running on port: 7860
